In [1]:
import json
import pandas as pd
import re
import random
import numpy as np
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

    return reviews

In [ ]:
def classify_it(filename, parametr): 
    i = 0
    reviews_lst_good= []
    reviews_lst_bad = []


    with open(filename, 'r') as f:
            for line in f:
                if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=50000:
                    reviews_lst_good.append (json.loads(line))

                elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=50000:
                    reviews_lst_bad.append (json.loads(line))

                if len(reviews_lst_good) + len (reviews_lst_bad) == 100000:
                    break
    # random elements in the list
    random.shuffle(reviews_lst_good)
    random.shuffle(reviews_lst_bad)  
    
    reviewerID_lst = []
    reviewerName_lst = []
    reviewText_lst = []
    overall_lst = []
    summary_lst = []

    # reviews_lst [0]['overall']<5 #['asin']for 
    
    if len (reviews_lst_bad) < len(reviews_lst_good):
        count_otz = len (reviews_lst_bad)
    else:
        count_otz = len (reviews_lst_good)

    for i in reviews_lst_good[:count_otz]:
#         reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
#         summary_lst.append (i['summary'])

    print ('Good: ',len(reviews_lst_good[:count_otz]))

    for i in reviews_lst_bad[:count_otz]:
#         reviewerID_lst.append (i['reviewerID'])
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
#         summary_lst.append (i['summary'])

    print ('Bad: ',len(reviews_lst_bad[:count_otz]))
    
    test_len = int(round (len(reviews_lst_bad[:count_otz])*0.1,0))
    
    print ('10%: ',test_len)

    df = pd.DataFrame ()

#     df['reviewerID'] = reviewerID_lst
    df['reviewText'] = reviewText_lst
    df['overall'] = overall_lst
#     df['summary'] = summary_lst
    
    df ['mark'] = 0
    df.loc[df['overall'] > 3,'mark'] = 1
    
    # 
    reviews_train = list(df[df['overall'] > 3]['reviewText'][:-test_len]) + list(df[df['overall'] < 3]['reviewText'][:-test_len])
    reviews_test = list(df[df['overall'] > 3]['reviewText'][-test_len:]) + list(df[df['overall'] < 3]['reviewText'][-test_len:])



    reviews_train_clean = preprocess_reviews(reviews_train)
    reviews_test_clean = preprocess_reviews(reviews_test)
    
    cv = TfidfVectorizer(binary=True)
    cv.fit(reviews_train_clean)
    X = cv.transform(reviews_train_clean)
    X_test = cv.transform(reviews_test_clean)
    
    y = list(df[df['overall'] > 3]['mark'][:-test_len]) + list(df[df['overall'] < 3]['mark'][:-test_len])
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, train_size = 0.75
    )
    
    print ('Подбор параметра для: ', filename)
    print ('    ')
    
    
    best_c_dict = {}
    main_dict = {}
    c_list = []
    result_1_list = []
    result_2_list = []
    
    grid = np.power(10.0, np.arange(-5, 6))
    
    for c in grid:
        lr = SVC(C=c)
        lr.fit(X_train, y_train)
        print ("Accuracy for C=%s: %s" 
               % (c, accuracy_score(y_val, lr.predict(X_val))))
        
        c_list.append (c)
        result_1_list.append (accuracy_score(y_val, lr.predict(X_val)))
        result_2_list.append (accuracy_score(list(df[df['overall'] > 3]['mark'][-test_len:]) + list(df[df['overall'] < 3]['mark'][-test_len:])
                            ,lr.predict(X_test)))
        
        
        best_c_dict [c] = accuracy_score(y_val, lr.predict(X_val))
        main_dict [c] = [filename, accuracy_score(y_val, lr.predict(X_val))]
        
    
    result = pd.DataFrame()
    result['c'] = c_list
    result ['result_1'] = result_1_list
    result ['result_2'] = result_2_list
    
    result.to_csv (filename+'_c.csv', index=False)
    
    inverse = [(value, key) for key, value in best_c_dict.items()]
    print (max(inverse)[1])

    final_model = LogisticRegression(C=best_c_dict[max(inverse)[1]])
    print ('Model use C = ', best_c_dict[max(inverse)[1]])
    final_model.fit(X,  y)
    print ("Final Accuracy: %s" 
           % accuracy_score(list(df[df['overall'] > 3]['mark'][-test_len:]) + list(df[df['overall'] < 3]['mark'][-test_len:])
                            , final_model.predict(X_test)))
    
    feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
        )
    }
    for best_positive in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1], 
        reverse=True)[:5]:
        print (best_positive)
    for best_negative in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1])[:5]:
        print (best_negative)
        
    del reviews_lst_good
    del reviews_lst_bad
    
    return main_dict

In [ ]:
import os
import gc

os.listdir('/Volumes/GoogleDrive/Мой диск/Work/Experiment')

for i in os.listdir('/Volumes/GoogleDrive/Мой диск/Work/Experiment'):
    if i[-5:] != 'ipynb' and i != '.ipynb_checkpoints' and i == 'Health_and_Personal_Care_5.json':
        print (i)
        print ('   ')
        dict_main = classify_it ('/Volumes/GoogleDrive/Мой диск/Work/Experiment/' + i, 0.25)
        gc.collect()
        

Health_and_Personal_Care_5.json
   
Good:  33300
Bad:  33300
10%:  3330


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Подбор параметра для:  /Volumes/GoogleDrive/Мой диск/Work/Experiment/Health_and_Personal_Care_5.json
    


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
